### Prototyping automatic validation checks for NTD reporting data: form RR-20
  
Decided *not* to use Pandera to validate this form. It is more straightforward and customizable to write our own functions.  
  
This notebook first imports a csv of cleaned data, presumably submitted from the RR-20 form, from the BlackCat report generation function on their website. We will assume that a future API will have this data in the **exact same format** as ther generated report has it in, which is what we requested from BlackCat developers. 
  
This notebook shows the development of the functions that are used in the executable file `rr20_check.py`

In [41]:
from google.cloud import bigquery
import pandas as pd
import pandera as pa
import numpy as np 
import datetime

#### FOR NOW:
For each org, filter to the rows with the latest `date_uploaded`. 
* Each revision that an agency submits into BlackCat should be in the `blackcat_raw` data table in BigQuery. By grabbing all rows with the latest date for each agency, we get the latest submitted report info.
* Since 2023 and 2022 data is kept in different tables, do this for each table and we will have the latest data submittal for each year.


In [42]:
this_year = datetime.datetime.now().year
last_year =(this_year-1)
print(last_year)

2022


In [37]:
### Query "For Now"
def get_bq_data(client, year, tablename):
    bq_data_query = f"""SELECT * FROM 
          (select *,
          RANK() OVER(PARTITION BY Organization_Legal_Name ORDER BY date_uploaded DESC) rank_date 
        from `cal-itp-data-infra.blackcat_raw.{year}_{tablename}`) s 
        WHERE rank_date = 1;
        """

    print(bq_data_query)
    
    df = client.query(bq_data_query).to_dataframe()
    df = df.drop_duplicates().drop(['rank_date', 'date_uploaded'], axis=1)
    return df


In [22]:
client = bigquery.Client()

In [43]:
rr20_service = get_bq_data(client, this_year, "rr20_service_data")
print(rr20_service.shape)
rr20_service.head()

,Organization_Legal_Name,Common_Name_Acronym_DBA,Fiscal_Year,Mode,Annual_VRM,Annual_VRH,Annual_UPT,Sponsored_UPT,VOMX
0,City of Rio Vista,None,2023,Demand Response (DR) - (PT),7622.0,1710.0,2402.0,0.0,1.0
1,City of Rio Vista,None,2023,Deviated Fixed Route (DF) - (PT),37147.0,1828.0,1680.0,0.0,1.0
2,Mountain Area Regional Transit Authority,None,2023,Bus (MB) (Fixed Route) - (DO),559275.0,39620.0,816510.0,0.0,18.0
3,Mountain Area Regional Transit Authority,None,2023,Commuter Bus (CB) - (DO),82420.0,3924.0,8627.0,0.0,4.0
4,Mountain Area Regional Transit Authority,None,2023,Demand Response (DR) - (DO),99309.0,8125.0,13730.0,0.0,9.0


In [31]:
rr20_exp_by_mode = get_bq_data(client, this_year, "rr20_expenses_by_mode")
print(rr20_exp_by_mode.shape)
rr20_exp_by_mode.head()

SELECT * FROM 
          (select *,
          RANK() OVER(PARTITION BY Organization_Legal_Name ORDER BY date_uploaded DESC) rank_date 
        from `cal-itp-data-infra.blackcat_raw.2023_rr20_expenses_by_mode`) s 
        WHERE rank_date = 1;
        
(146, 6)


,Organization_Legal_Name,Common_Name_Acronym_DBA,Fiscal_Year,Operating_Capital,Mode,Total_Annual_Expenses_By_Mode
0,Humboldt Transit Authority,HTA,2023,Capital,Commuter Bus (CB) - (DO),0
1,Humboldt Transit Authority,HTA,2023,Capital,Intercity Service (IC) - (DO),0
2,Humboldt Transit Authority,HTA,2023,Operating,Intercity Service (IC) - (DO),1237898
3,Humboldt Transit Authority,HTA,2023,Operating,Commuter Bus (CB) - (DO),3930557
4,Morongo Basin Transit Authority,None,2023,Capital,Commuter Bus (CB) - (DO),64601


In [32]:
orgs_q = """SELECT * FROM `cal-itp-data-infra.blackcat_raw.2023_organizations`"""
orgs = client.query(orgs_q).to_dataframe().drop_duplicates().drop(['date_uploaded'], axis=1)


print(orgs.shape)
orgs.head()

(90, 1)


,Organization
0,Alpine County Community Development
1,Amador Transit
2,Butte County Association of Governments/ Butte...
3,Calaveras Transit Agency
4,City of Arvin


In [2]:
# Older way - load from excel files
# rr20_exp_by_mode = pd.read_excel("../data/NTD_Annual_Report_Rural_2022.xlsx", 
#                      sheet_name="Expenses By Mode", index_col=None) 
# rr20_service = pd.read_excel("../data/NTD_Annual_Report_Rural_2022.xlsx", 
#                      sheet_name="Service Data", index_col=None) 
# orgs = pd.read_csv("../data/organizations.csv")

In [5]:
# data1 = rr20_service.merge(orgs, left_on ='Organization Legal Name', right_on = 'Organization', 
#                           indicator=True).query('_merge == "both"').drop(columns=['_merge', 'Organization'])
# print(data1.shape)
# data1.head()

(86, 9)


,Organization Legal Name,Common Name/Acronym/DBA,Fiscal Year,Mode,Annual VRM,Annual VRH,Annual UPT,Sponsored UPT,VOMX
0,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0
1,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0
2,Amador Transit,NaN,2022,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0
3,Amador Transit,NaN,2022,Deviated Fixed Route (DF) - (DO),153757.0,7302.0,16196.0,0.0,11.0
4,Calaveras Transit Agency,CTA,2022,Demand Response (DR) - (PT),23812.0,1416.0,2104.0,0.0,5.0


In [6]:
# Doubles the rows as expected - because of the 'Operating/Capital' column
# data = data1.merge(rr20_exp_by_mode, on = ['Organization Legal Name', 'Common Name/Acronym/DBA', 'Fiscal Year', 'Mode'])

# print(data.shape)
# data.head()

(172, 11)


,Organization Legal Name,Common Name/Acronym/DBA,Fiscal Year,Mode,Annual VRM,Annual VRH,Annual UPT,Sponsored UPT,VOMX,Operating/Capital,Total Annual Expenses By Mode
0,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Capital,0.0
1,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Operating,75944.0
2,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Capital,0.0
3,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Operating,208171.0
4,Amador Transit,NaN,2022,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0,Capital,0.0


In [57]:
# try the 2 merges together
data = rr20_service.merge(orgs, left_on ='Organization_Legal_Name', right_on = 'Organization', 
                          indicator=True).query('_merge == "both"').drop(columns=['_merge', 'Organization'])\
.merge(rr20_exp_by_mode, on = ['Organization_Legal_Name', 'Common_Name_Acronym_DBA', 'Fiscal_Year', 'Mode'])

In [58]:
print(data.shape)
data.head(20)

(146, 11)


,Organization_Legal_Name,Common_Name_Acronym_DBA,Fiscal_Year,Mode,Annual_VRM,Annual_VRH,Annual_UPT,Sponsored_UPT,VOMX,Operating_Capital,Total_Annual_Expenses_By_Mode
0,City of Rio Vista,None,2023,Demand Response (DR) - (PT),7622.0,1710.0,2402.0,0.0,1.0,Capital,0
1,City of Rio Vista,None,2023,Demand Response (DR) - (PT),7622.0,1710.0,2402.0,0.0,1.0,Operating,279300
2,City of Rio Vista,None,2023,Deviated Fixed Route (DF) - (PT),37147.0,1828.0,1680.0,0.0,1.0,Capital,0
3,City of Rio Vista,None,2023,Deviated Fixed Route (DF) - (PT),37147.0,1828.0,1680.0,0.0,1.0,Operating,298572
4,Mountain Area Regional Transit Authority,None,2023,Bus (MB) (Fixed Route) - (DO),559275.0,39620.0,816510.0,0.0,18.0,Capital,2311662
5,Mountain Area Regional Transit Authority,None,2023,Bus (MB) (Fixed Route) - (DO),559275.0,39620.0,816510.0,0.0,18.0,Operating,4140294
6,Mountain Area Regional Transit Authority,None,2023,Commuter Bus (CB) - (DO),82420.0,3924.0,8627.0,0.0,4.0,Capital,340668
7,Mountain Area Regional Transit Authority,None,2023,Commuter Bus (CB) - (DO),82420.0,3924.0,8627.0,0.0,4.0,Operating,483292
8,Mountain Area Regional Transit Authority,None,2023,Demand Response (DR) - (DO),99309.0,8125.0,13730.0,0.0,9.0,Capital,410476
9,Mountain Area Regional Transit Authority,None,2023,Demand Response (DR) - (DO),99309.0,8125.0,13730.0,0.0,9.0,Operating,874019


#### Get data for last year - 2022
 2022 data was only uploaded once so has slightly different schema

In [40]:
bq_2022_query = f"""SELECT * FROM `cal-itp-data-infra.blackcat_raw.{last_year}_rr20_service_data`"""
rr20_service_lastyr = client.query(bq_2022_query).to_dataframe().drop_duplicates()
print(rr20_service_lastyr.shape)
rr20_service_lastyr.head()

(89, 9)


,Organization_Legal_Name,Common_Name_Acronym_DBA,Fiscal_Year,Mode,Annual_VRM,Annual_VRH,Annual_UPT,Sponsored_UPT,VOMX
0,County of Siskiyou,CTA,2022,Bus (MB) (Fixed Route) - (DO),226547.0,10279.0,27970.0,0.0,5.0
1,"County of Nevada Public Works, Transit Service...",CTA,2022,Bus (MB) (Fixed Route) - (DO),322267.0,17926.0,99321.0,0.0,7.0
2,Mountain Area Regional Transit Authority,CTA,2022,Demand Response (DR) - (DO),101401.0,8316.0,12232.0,0.0,8.0
3,County of Mariposa,CTA,2022,Demand Response (DR) - (DO),117623.0,3577.0,8111.0,1904.0,8.0
4,"County of Nevada Public Works, Transit Service...",CTA,2022,Demand Response (DR) - (PT),103930.0,8505.0,18057.0,0.0,10.0


In [44]:
exp_2022_query = f"""SELECT * FROM `cal-itp-data-infra.blackcat_raw.{last_year}_rr20_expenses_by_mode`"""
rr20_exp_by_mode_lastyr = client.query(exp_2022_query).to_dataframe().drop_duplicates()
print(rr20_exp_by_mode_lastyr.shape)
rr20_exp_by_mode_lastyr.head()

(178, 6)


,Organization_Legal_Name,Common_Name_Acronym_DBA,Fiscal_Year,Operating_Capital,Mode,Total_Annual_Expenses_By_Mode
0,Amador Transit,None,2022,Operating,Commuter Bus (CB) - (DO),208171.0
1,Amador Transit,None,2022,Capital,Commuter Bus (CB) - (DO),0.0
2,Humboldt Transit Authority,HTA,2022,Operating,Commuter Bus (CB) - (DO),3414537.0
3,Humboldt Transit Authority,HTA,2022,Capital,Commuter Bus (CB) - (DO),138173.0
4,Morongo Basin Transit Authority,None,2022,Capital,Commuter Bus (CB) - (DO),100818.0


In [51]:
data_lastyear = (rr20_service_lastyr.merge(orgs, left_on ='Organization_Legal_Name', right_on = 'Organization', 
                            indicator=True).query('_merge == "both"').drop(columns=['_merge', 'Organization'])
                            .merge(rr20_exp_by_mode_lastyr, on = ['Organization_Legal_Name', 'Fiscal_Year', 'Mode'])
                .sort_values(by="Organization_Legal_Name"))
print(data_lastyear.shape)
data_lastyear.head()

(166, 12)


,Organization_Legal_Name,Common_Name_Acronym_DBA_x,Fiscal_Year,Mode,Annual_VRM,Annual_VRH,Annual_UPT,Sponsored_UPT,VOMX,Common_Name_Acronym_DBA_y,Operating_Capital,Total_Annual_Expenses_By_Mode
30,Alpine County Community Development,CTA,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,None,Operating,75944.0
31,Alpine County Community Development,CTA,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,None,Capital,0.0
24,Amador Transit,CTA,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,None,Operating,208171.0
25,Amador Transit,CTA,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,None,Capital,0.0
26,Amador Transit,eTrans,2022,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0,None,Capital,0.0


In [54]:
# 2022: we use the "Common Name" from the service data, if empty then from the expenses table. If neither empty, still use from the service data
data_lastyear['Common_Name_Acronym_DBA'] = data_lastyear['Common_Name_Acronym_DBA_x'].combine_first(data_lastyear['Common_Name_Acronym_DBA_y'])
data_lastyear.drop(columns=['Common_Name_Acronym_DBA_x', 'Common_Name_Acronym_DBA_y'], inplace=True)
data_lastyear.head()


,Organization_Legal_Name,Fiscal_Year,Mode,Annual_VRM,Annual_VRH,Annual_UPT,Sponsored_UPT,VOMX,Operating_Capital,Total_Annual_Expenses_By_Mode,Common_Name_Acronym_DBA
30,Alpine County Community Development,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Operating,75944.0,CTA
31,Alpine County Community Development,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Capital,0.0,CTA
24,Amador Transit,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Operating,208171.0,CTA
25,Amador Transit,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Capital,0.0,CTA
26,Amador Transit,2022,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0,Capital,0.0,eTrans


In [59]:
allyears = pd.concat([data, data_lastyear], ignore_index = True)
print(allyears.shape)
allyears.head()

(312, 11)


,Organization_Legal_Name,Common_Name_Acronym_DBA,Fiscal_Year,Mode,Annual_VRM,Annual_VRH,Annual_UPT,Sponsored_UPT,VOMX,Operating_Capital,Total_Annual_Expenses_By_Mode
0,City of Rio Vista,None,2023,Demand Response (DR) - (PT),7622.0,1710.0,2402.0,0.0,1.0,Capital,0.0
1,City of Rio Vista,None,2023,Demand Response (DR) - (PT),7622.0,1710.0,2402.0,0.0,1.0,Operating,279300.0
2,City of Rio Vista,None,2023,Deviated Fixed Route (DF) - (PT),37147.0,1828.0,1680.0,0.0,1.0,Capital,0.0
3,City of Rio Vista,None,2023,Deviated Fixed Route (DF) - (PT),37147.0,1828.0,1680.0,0.0,1.0,Operating,298572.0
4,Mountain Area Regional Transit Authority,None,2023,Bus (MB) (Fixed Route) - (DO),559275.0,39620.0,816510.0,0.0,18.0,Capital,2311662.0


#### Initial development of validation checks before the 2023 NTD reporting season started
Used 2022 and 2021 data.  
Commenting out these cells since they are no longer needed. However they show how things were developed.  
  
---  
To get data for 2021 - THIS IS FAKE - made some fake data
If we click "generate report" for 2021, RR-20 rural on BlackCat, we get an empty report. For the sake of time I am just making up fake data for 2021 to build the function to compare the prior year to this year.  
  
(scroll to bottom of notebook for code on how I made the fake data)

In [5]:
# exp_by_mode_2021 = pd.read_excel("../data/NTD_Annual_Report_Rural_2021.xlsx", 
#                      sheet_name="Expenses By Mode", index_col=None) 
# service_2021 = pd.read_excel("../data/NTD_Annual_Report_Rural_2021.xlsx", 
#                      sheet_name="Service Data", index_col=None) 

In [6]:
# all2021 = service_2021.merge(exp_by_mode_2021, on = ['Organization Legal Name', 'Common Name/Acronym/DBA', 'Fiscal Year', 'Mode'])

In [7]:
####------- Combine 2022 (rea) and 2021 (fake) data
# allyears = data.append(all2021, ignore_index = True)
# print(allyears.shape)
# allyears.head()

(344, 11)


/var/folders/3h/9n_2y6x178l41fsp4dg802f00000gn/T/ipykernel_67813/2063531429.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  allyears = data.append(all2021, ignore_index = True)


,Organization Legal Name,Common Name/Acronym/DBA,Fiscal Year,Mode,Annual VRM,Annual VRH,Annual UPT,Sponsored UPT,VOMX,Operating/Capital,Total Annual Expenses By Mode
0,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Capital,0.0
1,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Operating,75944.0
2,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Capital,0.0
3,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Operating,208171.0
4,Amador Transit,NaN,2022,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0,Capital,0.0


#### Add in cost per hr to df 

**Cost per hour (CPH)** includes, importantly, comparing it to the previous year.  
CPH = `Expenses on operations by mode/VRH`
  
Example NTD error message: `The calculated cost per hour for {MB - PT} equals {49.12}. The prior year’s calculated value equals {98.56}. This is a change of {-50.166} Percent.`

In [8]:
# https://stackoverflow.com/questions/59865432/can-i-use-pandas-dataframe-assign-with-a-variable-name
# Making df.assign accept variables by pasing in a dict

def make_ratio_cols(df, numerator, denominator, col_name, operation="sum"):
    if col_name is not None:
        # If a user specify a column name, use it
        # Raise error if the column already exists
        if col_name in df.columns:
            raise ValueError(f"Dataframe already has column '{col_name}'")
        else:
            _col_name = col_name
    
    if operation == "sum":    
        df = (df.groupby(['Organization Legal Name','Mode', 'Fiscal Year'])
              .apply(lambda x: x.assign(**{_col_name:
                     lambda x: x[numerator].sum() / x[denominator]}))
                    )
    else:
        df = (df.groupby(['Organization Legal Name','Mode', 'Fiscal Year'])
              .apply(lambda x: x.assign(**{_col_name:
                     lambda x: x[numerator] / x[denominator]}))
                    )
        
    return df

In [9]:
### testing
# 'Annual VRH' in allyears.columns
allyears = make_ratio_cols(allyears, 'Total Annual Expenses By Mode',
                       'Annual VRH', 
                       'cost_per_hr')


# test['cost_per_hr'] = (allyears.groupby(['Organization Legal Name','Mode', 'Fiscal Year'])
#                        .apply(lambda x: x['Total Annual Expenses By Mode'].sum() / x['Annual VRH']))
allyears.head(3)

,Organization Legal Name,Common Name/Acronym/DBA,Fiscal Year,Mode,Annual VRM,Annual VRH,Annual UPT,Sponsored UPT,VOMX,Operating/Capital,Total Annual Expenses By Mode,cost_per_hr
0,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Capital,0.0,118.108865
1,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Operating,75944.0,118.108865
2,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Capital,0.0,127.166158


In [115]:
## Alternate way to do it - 
# without a function, 1-liner to add column

# test2 = (allyears.groupby(['Organization Legal Name','Mode', 'Fiscal Year'])
#  .apply(lambda x: x.assign(cost_per_hr=lambda x: x['Total Annual Expenses By Mode'].sum() / x['Annual VRH']))
# )

# test2.head()

,Organization Legal Name,Common Name/Acronym/DBA,Fiscal Year,Mode,Annual VRM,Annual VRH,Annual UPT,Sponsored UPT,VOMX,Operating/Capital,Total Annual Expenses By Mode,cost_per_hr
0,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Capital,0.0,118.108865
1,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Operating,75944.0,118.108865
2,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Capital,0.0,127.166158
3,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Operating,208171.0,127.166158
4,Amador Transit,NaN,2022,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0,Capital,0.0,69.624206


#### Also add in Miles per vehicle: VRM/vehicles of max service (VOMS)  
Example NTD error message: `The calculated miles per vehicle for {DR - DO} is {5,221.40}. The prior year’s calculated value is {3,678.33}. This is a {41.95}% {increase} caused by a change in Vehicle Revenue Miles, the number of Vehicles Operated in Annual Maximum Service, or both.`

In [10]:
allyears = make_ratio_cols(allyears, 'Annual VRM', 'VOMX', 'miles_per_veh')

## Alternate way to do it - 
# allyears = (allyears.groupby(['Organization Legal Name','Mode', 'Fiscal Year'])
#  .apply(lambda x: x.assign(miles_per_veh=lambda x: x['Annual VRM'].sum() / x['VOMX']))
# )

allyears.head()

,Organization Legal Name,Common Name/Acronym/DBA,Fiscal Year,Mode,Annual VRM,Annual VRH,Annual UPT,Sponsored UPT,VOMX,Operating/Capital,Total Annual Expenses By Mode,cost_per_hr,miles_per_veh
0,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Capital,0.0,118.108865,20772.0
1,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Operating,75944.0,118.108865,20772.0
2,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Capital,0.0,127.166158,90944.0
3,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Operating,208171.0,127.166158,90944.0
4,Amador Transit,NaN,2022,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0,Capital,0.0,69.624206,12534.8


**Add in:**
* Fare Revenues per unlinked passenger trip
* Revenue Speed
* Trips per hr

In [11]:
allyears = make_ratio_cols(allyears, 'Total Annual Expenses By Mode', 'Annual UPT', 
                           'fare_rev_per_trip')  
allyears.head(3)

,Organization Legal Name,Common Name/Acronym/DBA,Fiscal Year,Mode,Annual VRM,Annual VRH,Annual UPT,Sponsored UPT,VOMX,Operating/Capital,Total Annual Expenses By Mode,cost_per_hr,miles_per_veh,fare_rev_per_trip
0,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Capital,0.0,118.108865,20772.0,197.770833
1,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Operating,75944.0,118.108865,20772.0,197.770833
2,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Capital,0.0,127.166158,90944.0,127.790669


In [12]:
# This calculation is NOT summing across modes like the ones above that involve total Expenses.
# Here we want to just calculate a fraction using the values in one row only. 
# So we add in 'operation = "mean" as a function argument. The function then does not sum across the numerator column.'
allyears = make_ratio_cols(allyears, 'Annual VRM', 'Annual VRH', 'rev_speed', operation = "mean")
allyears.head()

,Organization Legal Name,Common Name/Acronym/DBA,Fiscal Year,Mode,Annual VRM,Annual VRH,Annual UPT,Sponsored UPT,VOMX,Operating/Capital,Total Annual Expenses By Mode,cost_per_hr,miles_per_veh,fare_rev_per_trip,rev_speed
0,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Capital,0.0,118.108865,20772.0,197.770833,16.152411
1,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Operating,75944.0,118.108865,20772.0,197.770833,16.152411
2,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Capital,0.0,127.166158,90944.0,127.790669,27.777642
3,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Operating,208171.0,127.166158,90944.0,127.790669,27.777642
4,Amador Transit,NaN,2022,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0,Capital,0.0,69.624206,12534.8,29.630905,10.477098


In [55]:
# hand calculate Alpine value to double check it worked as it should have. Confirmed.
10386/643

16.152410575427684

In [13]:
allyears = make_ratio_cols(allyears, 'Annual UPT', 'Annual VRH', 'trips_per_hr', operation = "mean")
allyears.head()

,Organization Legal Name,Common Name/Acronym/DBA,Fiscal Year,Mode,Annual VRM,Annual VRH,Annual UPT,Sponsored UPT,VOMX,Operating/Capital,Total Annual Expenses By Mode,cost_per_hr,miles_per_veh,fare_rev_per_trip,rev_speed,trips_per_hr
0,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Capital,0.0,118.108865,20772.0,197.770833,16.152411,0.597201
1,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Operating,75944.0,118.108865,20772.0,197.770833,16.152411,0.597201
2,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Capital,0.0,127.166158,90944.0,127.790669,27.777642,0.995113
3,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Operating,208171.0,127.166158,90944.0,127.790669,27.777642,0.995113
4,Amador Transit,NaN,2022,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0,Capital,0.0,69.624206,12534.8,29.630905,10.477098,2.349716


In [66]:
384/643

0.5972006220839814

### Function development

In [14]:
import datetime

# this_year = datetime.datetime.now().year
# print(this_year)
this_year = 2022 #for testing purposes
last_year = this_year - 1
print(last_year)


allyears['Fiscal Year'].unique()

agencies = data['Organization Legal Name'].unique()

2021


In [15]:
### for testing
agency = 'Amador Transit'
mode = 'Demand Response (DR) - (DO)'
allyears[allyears['Organization Legal Name'] == agency]


,Organization Legal Name,Common Name/Acronym/DBA,Fiscal Year,Mode,Annual VRM,Annual VRH,Annual UPT,Sponsored UPT,VOMX,Operating/Capital,Total Annual Expenses By Mode,cost_per_hr,miles_per_veh,fare_rev_per_trip,rev_speed,trips_per_hr
2,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Capital,0.0,127.166158,90944.000000,127.790669,27.777642,0.995113
3,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Operating,208171.0,127.166158,90944.000000,127.790669,27.777642,0.995113
4,Amador Transit,NaN,2022,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0,Capital,0.0,69.624206,12534.800000,29.630905,10.477098,2.349716
5,Amador Transit,NaN,2022,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0,Operating,208246.0,69.624206,12534.800000,29.630905,10.477098,2.349716
6,Amador Transit,NaN,2022,Deviated Fixed Route (DF) - (DO),153757.0,7302.0,16196.0,0.0,11.0,Capital,0.0,134.220898,27955.818182,60.513769,21.056834,2.218022
7,Amador Transit,NaN,2022,Deviated Fixed Route (DF) - (DO),153757.0,7302.0,16196.0,0.0,11.0,Operating,980081.0,134.220898,27955.818182,60.513769,21.056834,2.218022
174,Amador Transit,NaN,2021,Commuter Bus (CB) - (DO),40000.0,1000.0,1629.0,0.0,1.0,Capital,0.0,208.171000,80000.000000,127.790669,40.000000,1.629000
175,Amador Transit,NaN,2021,Commuter Bus (CB) - (DO),40000.0,1000.0,1629.0,0.0,1.0,Operating,208171.0,208.171000,80000.000000,127.790669,40.000000,1.629000
176,Amador Transit,NaN,2021,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0,Capital,0.0,69.624206,12534.800000,29.630905,10.477098,2.349716
177,Amador Transit,NaN,2021,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0,Operating,208246.0,69.624206,12534.800000,29.630905,10.477098,2.349716


In [31]:
variable = 'cost_per_hr'
(allyears[(allyears['Organization Legal Name'] == agency) 
          & (allyears['Mode']==mode)
        & (allyears['Fiscal Year'] == last_year)]
 [variable].unique().sum()) #[0])

69.6242059511869

In [26]:
(allyears[(allyears['Organization Legal Name'] == agency) 
        & (allyears['Fiscal Year'] == this_year)]['Mode'].unique()
)

array(['Commuter Bus (CB) - (DO)', 'Demand Response (DR) - (DO)',
       'Deviated Fixed Route (DF) - (DO)'], dtype=object)

In [82]:
df = allyears.copy()

this_year = 2022 #for testing purposes
last_year = this_year - 1

def rr20_ratios(df, variable, threshold, this_year, last_year):
    agencies = df['Organization Legal Name'].unique()
    output = []
    for agency in agencies:

        if len(df[df['Organization Legal Name']==agency]) > 0:
        # Check whether data for both years is present, if so perform prior yr comparison.
            if (len(df[(df['Organization Legal Name']==agency) & (df['Fiscal Year']==this_year)]) > 0) \
                & (len(df[(df['Organization Legal Name']==agency) & (df['Fiscal Year']==last_year)]) > 0): 

                for mode in df[df['Organization Legal Name'] == agency]['Mode'].unique():
                    value_thisyr = (round(df[(df['Organization Legal Name'] == agency) 
                                          & (df['Mode']==mode)
                                          & (df['Fiscal Year'] == this_year)]
                                  [variable].unique()[0], 2))
                    value_lastyr = (round(df[(df['Organization Legal Name'] == agency) 
                                          & (df['Mode']==mode)
                                          & (df['Fiscal Year'] == last_year)]
                                  [variable].unique()[0], 2))
                    if (value_lastyr == 0) and (abs(value_thisyr - value_lastyr) >= threshold):
                        result = "fail"
                        check_name = f"{variable}"
                        mode = mode
                        description = (f"The {variable} for {mode} has changed from last year by > = {threshold*100}%, please provide a narrative justification.")
                    elif abs((value_lastyr - value_thisyr)/value_lastyr) >= threshold:
                        result = "fail"
                        check_name = f"{variable}"
                        mode = mode
                        description = (f"The {variable} for {mode} has changed from last year by {round((1 - value_thisyr/value_lastyr)*100, 1)}%, please provide a narrative justification.")
                    else:
                        result = "pass"
                        check_name = f"{variable}"
                        mode = mode
                        description = ""

                    output_line = {"Organization": agency,
                           "name_of_check" : check_name,
                                   "mode": mode,
                            "value_checked": f"{this_year} = {value_thisyr}, {last_year} = {value_lastyr}",
                            "pct_change": round(abs((value_lastyr - value_thisyr)/value_lastyr)*100, 1),
                            "check_status": result,
                            "Description": description}
                    output.append(output_line)
        else:
            print(f"There is no data for {agency}")
    checks = pd.DataFrame(output).sort_values(by="Organization")
    return checks

In [86]:
cph_checks = rr20_ratios(allyears, 'cost_per_hr', .3, this_year, last_year)
mpv_checks = rr20_ratios(allyears, 'miles_per_veh', .3, this_year, last_year)
fare_rev_checks = rr20_ratios(allyears, 'fare_rev_per_trip', .25, this_year, last_year)
rev_speed_checks = rr20_ratios(allyears, 'rev_speed', .15, this_year, last_year)
tph_checks = rr20_ratios(allyears, 'trips_per_hr', .30, this_year, last_year)



(86, 7)


,Organization,name_of_check,mode,value_checked,pct_change,check_status,Description
0,Alpine County Community Development,Annual VRM,Demand Response (DR) - (DO),"2022 = 10386.0, 2021 = 12386.0",16.1,pass,
1,Amador Transit,Annual VRM,Commuter Bus (CB) - (DO),"2022 = 45472.0, 2021 = 40000.0",13.7,pass,
2,Amador Transit,Annual VRM,Demand Response (DR) - (DO),"2022 = 31337.0, 2021 = 31337.0",0.0,pass,
3,Amador Transit,Annual VRM,Deviated Fixed Route (DF) - (DO),"2022 = 153757.0, 2021 = 153757.0",0.0,pass,
4,Calaveras Transit Agency,Annual VRM,Demand Response (DR) - (PT),"2022 = 23812.0, 2021 = 40000.0",40.5,fail,The Annual VRM for Demand Response (DR) - (PT)...


In [87]:
##---------------- Uncomment the following line pairs one by one to inspect the results
# print(cph_checks.shape)
# cph_checks.head()

print(mpv_checks.shape)
mpv_checks.head()

# print(fare_rev_checks.shape)
# fare_rev_checks.head()

# print(rev_speed_checks.shape)
# rev_speed_checks.head()

# print(tph_checks.shape)
# tph_checks.head()

(86, 7)


,Organization,name_of_check,mode,value_checked,pct_change,check_status,Description
0,Alpine County Community Development,Annual VRM,Demand Response (DR) - (DO),"2022 = 10386.0, 2021 = 12386.0",16.1,pass,
1,Amador Transit,Annual VRM,Commuter Bus (CB) - (DO),"2022 = 45472.0, 2021 = 40000.0",13.7,pass,
2,Amador Transit,Annual VRM,Demand Response (DR) - (DO),"2022 = 31337.0, 2021 = 31337.0",0.0,pass,
3,Amador Transit,Annual VRM,Deviated Fixed Route (DF) - (DO),"2022 = 153757.0, 2021 = 153757.0",0.0,pass,
4,Calaveras Transit Agency,Annual VRM,Demand Response (DR) - (PT),"2022 = 23812.0, 2021 = 40000.0",40.5,fail,The Annual VRM for Demand Response (DR) - (PT)...


In [91]:
rr20_checks = pd.concat([cph_checks, mpv_checks, fare_rev_checks, rev_speed_checks, tph_checks]).sort_values(by="Organization")

In [92]:
print(rr20_checks.shape)
rr20_checks.head(10)

(430, 7)


,Organization,name_of_check,mode,value_checked,pct_change,check_status,Description
0,Alpine County Community Development,cost_per_hr,Demand Response (DR) - (DO),"2022 = 118.11, 2021 = 118.11",0.0,pass,
0,Alpine County Community Development,trips_per_hr,Demand Response (DR) - (DO),"2022 = 0.6, 2021 = 0.6",0.0,pass,
0,Alpine County Community Development,rev_speed,Demand Response (DR) - (DO),"2022 = 16.15, 2021 = 19.26",16.1,fail,The rev_speed for Demand Response (DR) - (DO) ...
0,Alpine County Community Development,fare_rev_per_trip,Demand Response (DR) - (DO),"2022 = 197.77, 2021 = 197.77",0.0,pass,
0,Alpine County Community Development,miles_per_veh,Demand Response (DR) - (DO),"2022 = 20772.0, 2021 = 24772.0",16.1,pass,
1,Amador Transit,rev_speed,Commuter Bus (CB) - (DO),"2022 = 27.78, 2021 = 40.0",30.6,fail,The rev_speed for Commuter Bus (CB) - (DO) has...
3,Amador Transit,fare_rev_per_trip,Deviated Fixed Route (DF) - (DO),"2022 = 60.51, 2021 = 60.51",0.0,pass,
2,Amador Transit,fare_rev_per_trip,Demand Response (DR) - (DO),"2022 = 29.63, 2021 = 29.63",0.0,pass,
3,Amador Transit,miles_per_veh,Deviated Fixed Route (DF) - (DO),"2022 = 27955.82, 2021 = 27955.82",0.0,pass,
2,Amador Transit,miles_per_veh,Demand Response (DR) - (DO),"2022 = 12534.8, 2021 = 12534.8",0.0,pass,


In [93]:
rr20_checks.to_excel("../data/test_rr20.xlsx", index=False)

#### Function to check whether the VRM changed significantly.
Unlike the above metrics, VRM is not a ratio - the NTD error is just comparing a single number to it's value from the previous year.  
  
Example NTD error message: `For {DR - DO|DR - PT|CB - DO| CB - PT|MB - PT|MB - DO}, Annual Vehicle Revenue Miles {} {} percent compared to last year from {} to {}. This change is high.`

In [107]:
def check_single_number(df, variable, this_year, last_year, threshold=None):
    agencies = df['Organization Legal Name'].unique()
    output = []
    for agency in agencies:

        if len(df[df['Organization Legal Name']==agency]) > 0:
        # Check whether data for both years is present, if so perform prior yr comparison.
            if (len(df[(df['Organization Legal Name']==agency) & (df['Fiscal Year']==this_year)]) > 0) \
                & (len(df[(df['Organization Legal Name']==agency) & (df['Fiscal Year']==last_year)]) > 0): 

                for mode in df[df['Organization Legal Name'] == agency]['Mode'].unique():
                    value_thisyr = (round(df[(df['Organization Legal Name'] == agency) 
                                          & (df['Mode']==mode)
                                          & (df['Fiscal Year'] == this_year)]
                                  [variable].unique()[0], 2))
                    value_lastyr = (round(df[(df['Organization Legal Name'] == agency) 
                                          & (df['Mode']==mode)
                                          & (df['Fiscal Year'] == last_year)]
                                  [variable].unique()[0], 2))
                    
                    if (round(value_thisyr)==0 and round(value_lastyr) != 0) | (round(value_thisyr)!=0 and round(value_lastyr) == 0):
                        result = "fail"
                        check_name = f"{variable}"
                        mode = mode
                        description = (f"The {variable} for {mode} has changed either from or to zero compared to last year. Please provide a narrative justification.")
                    # run only the above check on whether something changed from zero to non-zero, if no threshold is given
                    elif threshold==None:
                        result = "pass"
                        check_name = f"{variable}"
                        mode = mode
                        description = ""
                        pass
                    # also check for pct change, if a threshold parameter is passed into function
                    elif (value_lastyr == 0) and (abs(value_thisyr - value_lastyr) >= threshold):
                        result = "fail"
                        check_name = f"{variable}"
                        mode = mode
                        description = (f"The {variable} for {mode} was 0 last year and has changed by > = {threshold*100}%, please provide a narrative justification.")
                    elif abs((value_lastyr - value_thisyr)/value_lastyr) >= threshold:
                        result = "fail"
                        check_name = f"{variable}"
                        mode = mode
                        description = (f"The {variable} for {mode} has changed from last year by {round(abs((value_lastyr - value_thisyr)/value_lastyr)*100, 1)}%; please provide a narrative justification.")                        
                    else:
                        result = "pass"
                        check_name = f"{variable}"
                        mode = mode
                        description = ""

                    output_line = {"Organization": agency,
                           "name_of_check" : check_name,
                                   "mode": mode,
                            "value_checked": f"{this_year} = {value_thisyr}, {last_year} = {value_lastyr}",
                            "check_status": result,
                            "Description": description}
                    output.append(output_line)
        else:
            print(f"There is no data for {agency}")
    checks = pd.DataFrame(output).sort_values(by="Organization")
    return checks

In [109]:
numeric_columns = allyears.select_dtypes(include=['number']).columns
allyears[numeric_columns] = allyears[numeric_columns].fillna(0)

vrm_checks = check_single_number(allyears,  'Annual VRM', this_year, last_year, threshold=.30)
vrm_checks

/var/folders/3h/9n_2y6x178l41fsp4dg802f00000gn/T/ipykernel_49221/1706745536.py:39: RuntimeWarning: invalid value encountered in double_scalars
  elif abs((value_lastyr - value_thisyr)/value_lastyr) >= threshold:


,Organization,name_of_check,mode,value_checked,check_status,Description
0,Alpine County Community Development,Annual VRM,Demand Response (DR) - (DO),"2022 = 10386.0, 2021 = 12386.0",pass,
1,Amador Transit,Annual VRM,Commuter Bus (CB) - (DO),"2022 = 45472.0, 2021 = 40000.0",pass,
2,Amador Transit,Annual VRM,Demand Response (DR) - (DO),"2022 = 31337.0, 2021 = 31337.0",pass,
3,Amador Transit,Annual VRM,Deviated Fixed Route (DF) - (DO),"2022 = 153757.0, 2021 = 153757.0",pass,
4,Calaveras Transit Agency,Annual VRM,Demand Response (DR) - (PT),"2022 = 23812.0, 2021 = 40000.0",fail,The Annual VRM for Demand Response (DR) - (PT)...
...,...,...,...,...,...,...
80,Town of Truckee,Annual VRM,Bus (MB) (Fixed Route) - (PT),"2022 = 101480.0, 2021 = 101480.0",pass,
82,Trinity County Department of Transportation,Annual VRM,Intercity Service (IC) - (DO),"2022 = 116976.0, 2021 = 116976.0",pass,
84,Tuolumne County Transit Agency (TCTA),Annual VRM,Demand Response (DR) - (PT),"2022 = 169446.0, 2021 = 169446.0",pass,
83,Tuolumne County Transit Agency (TCTA),Annual VRM,Bus (MB) (Fixed Route) - (PT),"2022 = 83834.0, 2021 = 83834.0",pass,


In [110]:
fare_rev_checks = check_single_number(allyears, 'fare_rev_per_trip', this_year, last_year)
fare_rev_checks

,Organization,name_of_check,mode,value_checked,check_status,Description
0,Alpine County Community Development,fare_rev_per_trip,Demand Response (DR) - (DO),"2022 = 197.77, 2021 = 197.77",pass,
1,Amador Transit,fare_rev_per_trip,Commuter Bus (CB) - (DO),"2022 = 127.79, 2021 = 127.79",pass,
2,Amador Transit,fare_rev_per_trip,Demand Response (DR) - (DO),"2022 = 29.63, 2021 = 29.63",pass,
3,Amador Transit,fare_rev_per_trip,Deviated Fixed Route (DF) - (DO),"2022 = 60.51, 2021 = 60.51",pass,
4,Calaveras Transit Agency,fare_rev_per_trip,Demand Response (DR) - (PT),"2022 = 99.93, 2021 = 99.93",pass,
...,...,...,...,...,...,...
80,Town of Truckee,fare_rev_per_trip,Bus (MB) (Fixed Route) - (PT),"2022 = 26.46, 2021 = 26.46",pass,
82,Trinity County Department of Transportation,fare_rev_per_trip,Intercity Service (IC) - (DO),"2022 = 116.08, 2021 = 116.08",pass,
84,Tuolumne County Transit Agency (TCTA),fare_rev_per_trip,Demand Response (DR) - (PT),"2022 = 53.24, 2021 = 53.24",pass,
83,Tuolumne County Transit Agency (TCTA),fare_rev_per_trip,Bus (MB) (Fixed Route) - (PT),"2022 = 28.09, 2021 = 28.09",pass,


In [111]:
voms0_check = check_single_number(allyears, 'VOMX', this_year, last_year)
voms0_check

,Organization,name_of_check,mode,value_checked,check_status,Description
0,Alpine County Community Development,VOMX,Demand Response (DR) - (DO),"2022 = 1.0, 2021 = 1.0",pass,
1,Amador Transit,VOMX,Commuter Bus (CB) - (DO),"2022 = 1.0, 2021 = 1.0",pass,
2,Amador Transit,VOMX,Demand Response (DR) - (DO),"2022 = 5.0, 2021 = 5.0",pass,
3,Amador Transit,VOMX,Deviated Fixed Route (DF) - (DO),"2022 = 11.0, 2021 = 11.0",pass,
4,Calaveras Transit Agency,VOMX,Demand Response (DR) - (PT),"2022 = 5.0, 2021 = 5.0",pass,
...,...,...,...,...,...,...
80,Town of Truckee,VOMX,Bus (MB) (Fixed Route) - (PT),"2022 = 3.0, 2021 = 3.0",pass,
82,Trinity County Department of Transportation,VOMX,Intercity Service (IC) - (DO),"2022 = 4.0, 2021 = 4.0",pass,
84,Tuolumne County Transit Agency (TCTA),VOMX,Demand Response (DR) - (PT),"2022 = 8.0, 2021 = 8.0",pass,
83,Tuolumne County Transit Agency (TCTA),VOMX,Bus (MB) (Fixed Route) - (PT),"2022 = 4.0, 2021 = 4.0",pass,


#### Check on any missing service data
NTD error: `One or more service data fields are are missing for {}: {}. You must report miles, hours, trips and vehicles operated in maximum service (VOMS) if the transit unit operated revenue service during the fiscal year.`

In [112]:
agency = 'Amador Transit'
mode = 'Demand Response (DR) - (DO)'
allyears[allyears['Organization Legal Name'] == agency]

,Organization Legal Name,Common Name/Acronym/DBA,Fiscal Year,Mode,Annual VRM,Annual VRH,Annual UPT,Sponsored UPT,VOMX,Operating/Capital,Total Annual Expenses By Mode,cost_per_hr,miles_per_veh,fare_rev_per_trip,rev_speed,trips_per_hr
2,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Capital,0.0,127.166158,90944.000000,127.790669,27.777642,0.995113
3,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Operating,208171.0,127.166158,90944.000000,127.790669,27.777642,0.995113
4,Amador Transit,NaN,2022,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0,Capital,0.0,69.624206,12534.800000,29.630905,10.477098,2.349716
5,Amador Transit,NaN,2022,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0,Operating,208246.0,69.624206,12534.800000,29.630905,10.477098,2.349716
6,Amador Transit,NaN,2022,Deviated Fixed Route (DF) - (DO),153757.0,7302.0,16196.0,0.0,11.0,Capital,0.0,134.220898,27955.818182,60.513769,21.056834,2.218022
7,Amador Transit,NaN,2022,Deviated Fixed Route (DF) - (DO),153757.0,7302.0,16196.0,0.0,11.0,Operating,980081.0,134.220898,27955.818182,60.513769,21.056834,2.218022
174,Amador Transit,NaN,2021,Commuter Bus (CB) - (DO),40000.0,1000.0,1629.0,0.0,1.0,Capital,0.0,208.171000,80000.000000,127.790669,40.000000,1.629000
175,Amador Transit,NaN,2021,Commuter Bus (CB) - (DO),40000.0,1000.0,1629.0,0.0,1.0,Operating,208171.0,208.171000,80000.000000,127.790669,40.000000,1.629000
176,Amador Transit,NaN,2021,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0,Capital,0.0,69.624206,12534.800000,29.630905,10.477098,2.349716
177,Amador Transit,NaN,2021,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0,Operating,208246.0,69.624206,12534.800000,29.630905,10.477098,2.349716


In [113]:
# first double-check how 0 vs blank cells are imported into pandas
test = pd.read_excel("~/Downloads/NTD_Annual_Report_Rural_2023_2023-09-28.xlsx", 
                     sheet_name="Service Data", index_col=None)
test

,Organization Legal Name,Common Name/Acronym/DBA,Fiscal Year,Mode,Annual VRM,Annual VRH,Annual UPT,Sponsored UPT,VOMX
0,City of Arvin,NaN,2023,Demand Response (DR) - (DO),12495.0,1988.0,4551.0,0.0,1.0
1,City of Arvin,NaN,2023,Deviated Fixed Route (DF) - (DO),120801.0,5499.0,55125.0,0.0,3.0
2,City of Corcoran - Corcoran Area Transit,NaN,2023,Demand Response (DR) - (DO),31978.0,3691.0,25199.0,0.0,4.0
3,County of Shasta Department of Public Works,NaN,2023,Bus (MB) (Fixed Route) - (PT),NaN,NaN,NaN,NaN,NaN
4,County of Shasta Department of Public Works,NaN,2023,Commuter Bus (CB) - (PT),NaN,NaN,NaN,NaN,NaN
5,Plumas County Transportation Commission,PCTC,2023,Deviated Fixed Route (DF) - (PT),196883.0,5929.0,24699.0,0.0,6.0
6,Tehama County Transit Agency,TRAX,2023,Bus (MB) (Fixed Route) - (PT),463998.0,20838.0,105515.0,0.0,7.0
7,Tehama County Transit Agency,TRAX,2023,Demand Response (DR) - (PT),114389.0,6626.0,14348.0,0.0,7.0


In [119]:
print(test.loc[3, 'Annual VRM'])
# pd.isnull(test.loc[3, 'Annual VRM']) #True
# pd.isna(test.loc[3, 'Annual VRM']) #True

print(test.loc[2, 'Sponsored UPT'])
pd.isnull(test.loc[2, 'Sponsored UPT']) #False for 0 values

nan
0.0


False

In [125]:
service_cols = ['Annual VRM', 'Annual VRH', "Annual UPT",
               "Sponsored UPT", "VOMX"]

mask = [test[x].isnull() for x in service_cols]
mask

[0    False
 1    False
 2    False
 3     True
 4     True
 5    False
 6    False
 7    False
 Name: Annual VRM, dtype: bool,
 0    False
 1    False
 2    False
 3     True
 4     True
 5    False
 6    False
 7    False
 Name: Annual VRH, dtype: bool,
 0    False
 1    False
 2    False
 3     True
 4     True
 5    False
 6    False
 7    False
 Name: Annual UPT, dtype: bool,
 0    False
 1    False
 2    False
 3     True
 4     True
 5    False
 6    False
 7    False
 Name: Sponsored UPT, dtype: bool,
 0    False
 1    False
 2    False
 3     True
 4     True
 5    False
 6    False
 7    False
 Name: VOMX, dtype: bool]

In [126]:
test[test[mask]]

KeyError: "None of [Index([(False, False, False, True, True, False, False, False),\n       (False, False, False, True, True, False, False, False),\n       (False, False, False, True, True, False, False, False),\n       (False, False, False, True, True, False, False, False),\n       (False, False, False, True, True, False, False, False)],\n      dtype='object')] are in the [columns]"

In [158]:
mask = test['Annual VRM'].isnull() | test['Annual VRH'].isnull() | test['Annual UPT'].isnull() | test['Annual UPT'].isnull() | test["VOMX"].isnull()
print(test[mask])

                       Organization Legal Name Common Name/Acronym/DBA  \
3  County of Shasta Department of Public Works                     NaN   
4  County of Shasta Department of Public Works                     NaN   

   Fiscal Year                           Mode  Annual VRM  Annual VRH  \
3         2023  Bus (MB) (Fixed Route) - (PT)         NaN         NaN   
4         2023       Commuter Bus (CB) - (PT)         NaN         NaN   

   Annual UPT  Sponsored UPT  VOMX  
3         NaN            NaN   NaN  
4         NaN            NaN   NaN  


In [162]:
orgs_missing_data = test[mask]['Organization Legal Name'].unique()
orgs_missing_data

array(['County of Shasta Department of Public Works'], dtype=object)

In [ ]:
output = []
for x in orgs_missing_data:
    result = "fail"
    check_name = "Missing service data check"
    mode = ""
    description = ("One or more service data values is missing in these columns. Please revise in BlackCat and resubmit.'Annual VRM', 'Annual VRH', 'Annual UPT',
               'Sponsored UPT', 'VOMX'")
    output_line = {"Organization": x,
                   "name_of_check" : check_name,
                   "mode": mode,
                    "value_checked": "Service data columns",
                    "check_status": result,
                    "Description": description}
            output.append(output_line)

In [170]:
# Test code to subtract lists from each other
orgs_missing_data = ['a', 'b', 'c']
all_orgs = ['a', 'b', 'c', 'd', 'e']
orgs_not_missing_data = list(set(all_orgs) - set(orgs_missing_data))
orgs_not_missing_data

['d', 'e']

In [184]:
def check_missing_servicedata(df):
    agencies = df['Organization Legal Name'].unique()
    
    mask = df['Annual VRM'].isnull() | df['Annual VRH'].isnull() | df['Annual UPT'].isnull() | df['Annual UPT'].isnull() | df["VOMX"].isnull()
    orgs_missing_data = df[mask]['Organization Legal Name'].unique()
    print(f"missing = {orgs_missing_data}")
    orgs_not_missing_data = list(set(agencies) - set(orgs_missing_data))
    print(f"Not missing = {orgs_not_missing_data}")
    
    output = []
    for x in agencies:
        if x in orgs_missing_data:
            result = "fail"
            check_name = "Missing service data check"
            mode = ""
            description = ("One or more service data values is missing in these columns. Please revise in BlackCat and resubmit.'Annual VRM', 'Annual VRH', 'Annual UPT','Sponsored UPT', 'VOMX'")
        elif x in orgs_not_missing_data:
            result = "pass"
            check_name = "Missing service data check"
            mode = ""
            description = ""
        output_line = {"Organization": x,
                    "name_of_check" : check_name,
                    "mode": mode,
                        "value_checked": "Service data columns",
                        "check_status": result,
                        "Description": description}
        output.append(output_line)
    checks = pd.DataFrame(output).sort_values(by="Organization")
    
    return checks

In [185]:
testcheck = check_missing_servicedata(test)
testcheck

missing = ['County of Shasta Department of Public Works']
Not missing = ['Plumas County Transportation Commission', 'City of Corcoran - Corcoran Area Transit', 'City of Arvin', 'Tehama County Transit Agency']


,Organization,name_of_check,mode,value_checked,check_status,Description
0,City of Arvin,Missing service data check,,Service data columns,pass,
1,City of Corcoran - Corcoran Area Transit,Missing service data check,,Service data columns,pass,
2,County of Shasta Department of Public Works,Missing service data check,,Service data columns,fail,One or more service data values is missing in ...
3,Plumas County Transportation Commission,Missing service data check,,Service data columns,pass,
4,Tehama County Transit Agency,Missing service data check,,Service data columns,pass,


In [155]:
agency = 'City of Arvin'
mode = 'Demand Response (DR) - (DO)'

test[(test['Organization Legal Name']==agency) & (test['Mode']==mode)]['Annual VRM'][0]

12495.0

In [156]:
## Not using this one - much less efficient.

# def check_missing_servicedata(df):
#     service_cols = ['Annual VRM', 'Annual VRH', "Annual UPT",
#                "Sponsored UPT", "VOMX"]
#     agencies = df['Organization Legal Name'].unique()
#     output = []
#     for agency in agencies:
#         if len(df[df['Organization Legal Name']==agency]) > 0:
#                 for mode in df[df['Organization Legal Name'] == agency]['Mode'].unique():
                    
#                     if (df[(df['Organization Legal Name']==agency) & (df['Mode']==mode)]['Annual VRM'][0].isnull()): 
#                     #| (df['Annual VRH'].isnull()) | (df['Annual UPT'].isnull()) | (df['Annual UPT'].isnull()) | (df["VOMX"].isnull()):
#                         nan_cols = [i for i in df.service_cols if df[i].isnull().any()]
#                         result = "fail"
#                         check_name = "Missing service data check"
#                         mode = mode
#                         description = (f"Service data is missing for {mode} in {nan_cols}. Please revise in BlackCat and resubmit.")
#                     else:
#                         result = "pass"
#                         check_name = "Missing service data check"
#                         mode = mode
#                         description = ""
                    
#                     output_line = {"Organization": agency,
#                            "name_of_check" : check_name,
#                                    "mode": mode,
#                             "value_checked": "Service data columns",
#                             "check_status": result,
#                             "Description": description}
#                     output.append(output_line)
#     checks = pd.DataFrame(output).sort_values(by="Organization")
#     return checks


---
  
The below cells show how I made the fake data - now commented out.

In [9]:
# fake2021_service = data1.copy()
# fake2021_service.head(3)

,Organization Legal Name,Common Name/Acronym/DBA,Fiscal Year,Mode,Annual VRM,Annual VRH,Annual UPT,Sponsored UPT,VOMX
0,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0
1,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0
2,Amador Transit,NaN,2022,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0


In [42]:
##----- Tried these ways of automatically generating fake data, but they take too much memory and crash

# import numpy as np

# def vrm(vrm):
#     return np.random.rand(int(vrm*0.9), int(vrm*1.1))


# fake2021_service['VRM'] = fake2021_service['Annual VRM'].apply(lambda x: vrm(x))

# fake2021_service.loc[50, 'Annual VRM'] = vrm(fake2021_service.loc[50, 'Annual VRM'])
# print(fake2021_service.loc[50, 'Annual VRM'] )

In [10]:
# fake2021_service.loc[0,'Annual VRM'] = 12386
# print(fake2021_service.loc[0,'Annual VRM'])
# fake2021_service.loc[1,'Annual VRM'] = 40000
# print(fake2021_service.loc[1,'Annual VRM'])
# fake2021_service.loc[4,'Annual VRM'] = 40000
# fake2021_service.loc[1,'Annual VRH'] = 1000
# print(fake2021_service.loc[1,'Annual VRH'])
# print(fake2021_service.loc[4,'Annual VRM'])

# fake2021_service.loc[9,'VOMX'] = 7
# print(fake2021_service.loc[9,'VOMX'])

# fake2021_service.loc[:,'Fiscal Year'] = 2021

12386.0
40000.0
1000.0
40000.0
7.0


In [11]:
# fake2021_service.head()

,Organization Legal Name,Common Name/Acronym/DBA,Fiscal Year,Mode,Annual VRM,Annual VRH,Annual UPT,Sponsored UPT,VOMX
0,Alpine County Community Development,NaN,2021,Demand Response (DR) - (DO),12386.0,643.0,384.0,274.0,1.0
1,Amador Transit,NaN,2021,Commuter Bus (CB) - (DO),40000.0,1000.0,1629.0,0.0,1.0
2,Amador Transit,NaN,2021,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0
3,Amador Transit,NaN,2021,Deviated Fixed Route (DF) - (DO),153757.0,7302.0,16196.0,0.0,11.0
4,Calaveras Transit Agency,CTA,2021,Demand Response (DR) - (PT),40000.0,1416.0,2104.0,0.0,5.0


In [12]:
# fake_exp = rr20_exp_by_mode.copy()
# fake_exp['Fiscal Year'] = 2021

In [157]:
### Export, for code development
## We will assume that the data coming in from the API will be *exactly* in this format

# with pd.ExcelWriter("../data/NTD_Annual_Report_Rural_2021_fake.xlsx") as writer:
    
#     fake2021_service.to_excel(writer, sheet_name="Expenses By Mode", index=False)
#     fake_exp.to_excel(writer, sheet_name="Service Data", index=False)


In [33]:
test2 = allyears.copy()
test2.head()

,Organization Legal Name,Common Name/Acronym/DBA,Fiscal Year,Mode,Annual VRM,Annual VRH,Annual UPT,Sponsored UPT,VOMX,Operating/Capital,Total Annual Expenses By Mode,cost_per_hr,miles_per_veh
0,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Capital,0.0,118.108865,20772.0
1,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Operating,75944.0,118.108865,20772.0
2,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Capital,0.0,127.166158,90944.0
3,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Operating,208171.0,127.166158,90944.0
4,Amador Transit,NaN,2022,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0,Capital,0.0,69.624206,12534.8


In [40]:
test2.loc[0,'Total Annual Expenses By Mode'] = 2000
test2.head(3)

,Organization Legal Name,Common Name/Acronym/DBA,Fiscal Year,Mode,Annual VRM,Annual VRH,Annual UPT,Sponsored UPT,VOMX,Operating/Capital,Total Annual Expenses By Mode,cost_per_hr,miles_per_veh
0,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Capital,2000.0,118.108865,20772.0
1,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Operating,75944.0,118.108865,20772.0
2,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Capital,0.0,127.166158,90944.0


In [41]:
test2 = (test2.groupby(['Organization Legal Name','Mode', 'Fiscal Year'])
 .apply(lambda x: x.assign(cost_per_hr=lambda x: x['Total Annual Expenses By Mode'].sum() / x['Annual VRH']))
)

test2.head()

,Organization Legal Name,Common Name/Acronym/DBA,Fiscal Year,Mode,Annual VRM,Annual VRH,Annual UPT,Sponsored UPT,VOMX,Operating/Capital,Total Annual Expenses By Mode,cost_per_hr,miles_per_veh
0,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Capital,2000.0,121.219285,20772.0
1,Alpine County Community Development,NaN,2022,Demand Response (DR) - (DO),10386.0,643.0,384.0,274.0,1.0,Operating,75944.0,121.219285,20772.0
2,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Capital,0.0,127.166158,90944.0
3,Amador Transit,NaN,2022,Commuter Bus (CB) - (DO),45472.0,1637.0,1629.0,0.0,1.0,Operating,208171.0,127.166158,90944.0
4,Amador Transit,NaN,2022,Demand Response (DR) - (DO),31337.0,2991.0,7028.0,315.0,5.0,Capital,0.0,69.624206,12534.8


In [42]:
(75944)/643.0


118.10886469673406

In [43]:
(75944 + 2000)/643.0

121.21928460342146